In [12]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import regex as re
import numpy as np
import urllib.parse ## get smiles url encoded.
import requests
import os
import subprocess

In [13]:
## smile ==> pdb using REST api.
smiles = "CCOS(=O)(=O)OCCOS(=O)(=O)[O-]"
rest_sdf_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{}/sdf?record_type=2d"
rest_name_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{}/record/JSON"
smiles_en = rest_sdf_url.format(urllib.parse.quote(smiles))

In [14]:
r = requests.get(smiles_en)

# Using RDKit

In [15]:
def smiles_to_iupac(sims):
    
    rest_name_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{}/record/JSON"
    smiles_en = rest_name_url.format(urllib.parse.quote(sims))
    resp = requests.get(smiles_en)
    resp_json = resp.json()
    return resp_json['PC_Compounds'][0]['props'][7]['value']['sval']


In [42]:
def smile_to_gauss(ion_name, sims):
    #ion_name = re.sub('[ ~!@#$%^&*()_,=-]', '_', ion_name)
    mol = Chem.MolFromSmiles(sims)
    m2 = Chem.AddHs(mol)
    AllChem.EmbedMolecule(m2)
    AllChem.MMFFOptimizeMolecule(m2)
    netcharge = Chem.GetFormalCharge(m2)
    gauss_body = re.split('\n\n', Chem.MolToXYZBlock(m2) ,1)[1]+'\n'
    Chem.PDBWriter("{}.pdb".format(ion_name)).write(m2)
    os.system("obabel {0}.pdb -O {0}.pdb --gen3d -h".format(ion_name))
    os.system("obabel -ipdb {0}_ob.pdb -omol2 > {0}.mol2".format(ion_name))
    gauss_header = '''%NProcShared=28
%mem=200GB
%Chk=job_{0}.chk
#p B3LYP/6-311G** em=GD3BJ Opt freq int=fine

{0}

{1}  1
'''.format(ion_name, netcharge)
    
    gauss_sps = """--link1--
%oldchk=./job_{0}.chk
%chk=job_{0}_gas.chk
# B3LYP/def2TZVP em=GD3BJ geom=allcheck


--link1--
%oldchk=./job_{0}.chk
%chk=job_{0}_solv.chk
# B3LYP/def2TZVP em=GD3BJ scrf=solvent geom=allcheck

""".format(ion_name)

    with open ("job_{}.com".format(ion_name), 'w') as fhandle:
        fhandle.writelines(gauss_header+gauss_body+gauss_sps)

In [57]:
def gen_submit(ion_name):
    sbatch_header = """#!/bin/bash 
#SBATCH -N 1
##SBATCH --time=24:00:00
#SBATCH --tasks-per-node=1
#SBATCH --cpus-per-task=28
#SBATCH --job-name={0}
#SBATCH -p short
#SBATCH --mem=200G
#SBATCH -t 1-00:00:00

module purge
module load gaussian/g16
source /shared/centos7/gaussian/g16/bsd/g16.profile
source /work/hung_group/xu.jiam/miniconda3/bin/activate  ## Load miniconda

export HOME_DIR=$SLURM_SUBMIT_DIR
export GAUSS_SCRDIR=$SLURM_SUBMIT_DIR

date > {0}.log
g16 job_{0}.com
date > {0}.log

formchk job_{0}.chk {0}.fch
formchk job_{0}_gas.chk vac_{0}.fch
formchk job_{0}_solv.chk sol_{0}.fch

cp /scratch/xu.jiam/multiwfn/settings.inp ./

/scratch/xu.jiam/multiwfn/Multiwfn vac_{0}.fch << EOF
7
18
1
y
q
EOF

/scratch/xu.jiam/multiwfn/Multiwfn sol_{0}.fch << EOF
7
18
1
y
q
EOF

python chg_swap.py --i {0}

oldpath=`pwd`
cp {0}_resp2.mol2 /scratch/xu.jiam/sobtop
cd /scratch/xu.jiam/sobtop
cp $oldpath/{0}.fch ./

./sobtop {0}_resp2.mol2 << EOF
2

1
2
3
{0}.fch



0
EOF

cp {0}_resp2.gro $oldpath
cp {0}_resp2.top $oldpath
cp {0}_resp2.itp $oldpath

cd $oldpath

python itp_combin.py --i {0}

module load openmpi/3.1.2
module load python/2.7.15
module load gromacs/2018.4-cpu
source /shared/centos7/gromacs/2018.4-cpu/bin/GMXRC

 
## if ! [ -f "ini_0.gro" ]; then
## mpirun -np 1 gmx_mpi insert-molecules -ci {0}_resp2.gro -nmol 1200 -box 10 10 10 -o ini_0.gro
## 
## else
## mpirun -np 1 gmx_mpi insert-molecules -f ini_0.gro -ci {0}_resp2.gro -nmol 1200 -box 10 10 10 -o ini.gro
## fi

""".format(ion_name)
    with open ("handin_{}.slurm".format(ion_name), "w") as fhandle:
        fhandle.write(sbatch_header.format(ion_name))

In [58]:
def top_file_gen(ion_names):
    
    if len(ion_names) == 3:
        top_file='''
[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ    fudgeQQ
     1              2              yes            0.5       0.8333

#include "gaff_nonbon.itp"
#include "gaff_bon.itp"
#include "{0}_resp2.itp"
#include "{1}_resp2.itp"
#include "{2}_resp2.itp"

[ system ]
resp2

[ molecules ]
; Molecule      nmols
{0}_resp2     1200
{1}_resp2    1200
{2}_resp2    1200'''.format(ion_names[0], ion_names[1], ion_names[2])
        with open('system.top', 'w') as fhandle:
            fhandle.writelines(top_file)
    
    elif len(ion_names) == 2:
        top_file='''
[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ    fudgeQQ
     1              2              yes            0.5       0.8333

#include "gaff_nonbon.itp"
#include "gaff_bon.itp"
#include "{0}_resp2.itp"
#include "{1}_resp2.itp"

[ system ]
resp2

[ molecules ]
; Molecule      nmols
{2}     1200
{3}    1200'''.format(long_name_to_itp_name(ion_names[0]), long_name_to_itp_name(ion_names[1]),
                      long_name_to_3_letters(ion_names[0]), long_name_to_3_letters(ion_names[1]))
        with open('system.top', 'w') as fhandle:
            fhandle.writelines(top_file)
            
    elif len(ion_names) == 1:
        top_file='''
[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ    fudgeQQ
     1              2              yes            0.5       0.8333

#include "{0}_resp2.itp"

[ system ]
resp2

[ molecules ]
; Molecule      nmols
{1}     1200'''.format(ion_names[0], re.findall('[a-zA-Z]+', ion_name)[0][:3].upper())
        with open('system.top', 'w') as fhandle:
            fhandle.writelines(top_file)
            

In [67]:
def loadgro():
    os.system("""
module load openmpi/3.1.2
module load python/2.7.15
module load gromacs/2018.4-cpu
source /shared/centos7/gromacs/2018.4-cpu/bin/GMXRC
""")

In [59]:
def long_name_to_3_letters(ion_name):
    return re.findall('[a-zA-Z]+', ion_name)[0][:3].upper()

def long_name_to_itp_name(ion_name):
    return re.findall('[a-zA-Z]+', ion_name)[0]

In [60]:
def array_to_list_for_write(array):
    rows_to_write = []
    for row in array:
        row_to_write = "    ".join(row)
        rows_to_write.append('{}\n'.format(row_to_write))
    return rows_to_write

In [61]:
#sims = ['CCCCCCN1C=C[N+](=C1)C', 'C(F)(F)(F)S(=NS(=O)(=O)C(F)(F)F)(=O)[O-]']


# itp cat

In [62]:
#ion_name = '1_hexyl_3_methylimidazol_3_ium'

In [63]:
def itp_unify(ion_name):

    index_to_atomtype = {}
    atomtype_bonds = []
    atomtype_angles = []
    atomtype_dihedrals = []

    index_atomtypes = []
    index_moleculetype = []
    index_atoms = []
    index_bonds = []
    index_angles = []
    index_dihedrals = []
    index_pairs = []

    with open ("{0}_resp2.itp".format(ion_name)) as ff:
        line = ff.readline()
        while line:
            if line.startswith('['):
                section = re.match('\[ \w+\ ]', line).captures()[0] #store the section header
                line = ff.readline()

                while line and not line.isspace(): #section ends with a blank line
                    if not line.startswith(';'):

                        data = line.split()

                        if 'atoms' in section:
                            index_to_atomtype[data[0]] = data[1]
                            index_atoms.append(line)

                        elif 'atomtypes' in section:
                            index_atomtypes.append(line)

                        elif 'moleculetype' in section:
                            index_moleculetype.append(line)

                        elif 'bonds' in section:
                            atom_i = index_to_atomtype[data[0]]
                            atom_j = index_to_atomtype[data[1]]
                            atomtype_bonds.append(np.hstack((atom_i, atom_j, data[2:5])))
                            index_bonds.append(np.hstack((data[:2])))

                        elif 'angles' in section:
                            atom_i = index_to_atomtype[data[0]]
                            atom_j = index_to_atomtype[data[1]]
                            atom_k = index_to_atomtype[data[2]]
                            atomtype_angles.append(np.hstack((atom_i, atom_j, atom_k, data[3:6])))
                            index_angles.append(np.hstack((data[:3])))

                        elif 'dihedrals' in section:
                            atom_i = index_to_atomtype[data[0]]
                            atom_j = index_to_atomtype[data[1]]
                            atom_k = index_to_atomtype[data[2]]
                            atom_l = index_to_atomtype[data[3]]
                            atomtype_dihedrals.append(np.hstack((atom_i, atom_j, atom_k, atom_l, data[4:8])))
                            index_dihedrals.append(np.hstack((data[:5])))

                        elif 'pairs' in section:
                            index_pairs.append(line)



                    line = ff.readline()
            line = ff.readline() 

    ## Removing redundant bad_types:

    bondtypes = []
    angletypes = []
    dihedraltypes = []

    for i in np.unique(np.array(atomtype_bonds)[:,:3], axis=0):
        a = np.where(np.sum(np.array(atomtype_bonds)[:,:3] == i, axis=1) == 3)
        bondtypes.append(np.hstack((i, np.round(np.average(np.array(np.array(atomtype_bonds)[a[0]][:, -2:], dtype=float), axis=0), 6))))
    for i in np.unique(np.array(atomtype_angles)[:,:4], axis=0):
        a = np.where(np.sum(np.array(atomtype_angles)[:,:4] == i, axis=1) == 4)
        angletypes.append(np.hstack((i, np.round(np.average(np.array(np.array(atomtype_angles)[a[0]][:, -2:], dtype=float), axis=0), 6))))   
    for i in np.unique(np.array(atomtype_dihedrals)[:,:5], axis=0):
        a = np.where(np.sum(np.array(atomtype_dihedrals)[:,:5] == i, axis=1) == 5)
        dihedraltypes.append(np.hstack((i, np.round(np.average(np.array(np.array(atomtype_dihedrals)[a[0]][:, -3:], dtype=float), axis=0), 4))))

    ## Exporting data

    bondtypes = array_to_list_for_write(bondtypes)
    angletypes = array_to_list_for_write(angletypes)
    dihedraltypes = array_to_list_for_write(dihedraltypes)

    with open ('{0}_resp2.itp'.format(re.findall('[a-zA-Z]+', ion_name)[0]), 'w') as fhandle:
        fhandle.write('[ moleculetype ]\n')
        fhandle.write('{}    3'.format(re.findall('[a-zA-Z]+', ion_name)[0][:3].upper()))

        fhandle.write('\n[ atoms ]\n')
        fhandle.writelines(index_atoms)

        fhandle.write('\n[ bonds ]\n')
        index_bonds = array_to_list_for_write(index_bonds)
        fhandle.writelines(index_bonds)

        fhandle.write('\n[ pairs ]\n')
        fhandle.writelines(index_pairs)

        fhandle.write('\n[ angles ]\n')
        index_angles = array_to_list_for_write(index_angles)
        fhandle.writelines(index_angles)

        fhandle.write('\n[ dihedrals ]\n')
        index_dihedrals = array_to_list_for_write(index_dihedrals)
        fhandle.writelines(index_dihedrals)

    ## Write bond infomation

    if os.path.exists('gaff_bon.itp'):
        with open("gaff_bon.itp") as fhandle:
            lines = fhandle.readlines()

        bad_line = [lines.index(i) for i in lines if 'bondtypes' in i or 'angletypes' in i or 'dihedraltypes' in i]
        lines.insert(bad_line[0]+1, "".join(bondtypes))
        lines.insert(bad_line[1]+len(bondtypes)+1, "".join(angletypes))
        lines.insert(bad_line[2]+len(bondtypes)+len(angletypes)+1, "".join(dihedraltypes))

        with open ("gaff_bon.itp",'w') as ff:
            ff.writelines(lines)

    else:
        with open("gaff_bon.itp", "w") as fhandle:    
            fhandle.write("[ bondtypes ]\n")
            fhandle.writelines(bondtypes)
            fhandle.write("\n[ angletypes ]\n")
            fhandle.writelines(angletypes)
            fhandle.write("\n[ dihedraltypes ]\n")
            fhandle.writelines(dihedraltypes)

    ## Write non-bond information
    if os.path.exists('gaff_nonbon.itp'):
        for line in index_atomtypes:
            os.system("echo '{}' >> gaff_nonbon.itp".format(line.strip()))

    else:
        with open('gaff_nonbon.itp', 'w') as fhandle:
            fhandle.write('[ atomtypes ]\n')
            fhandle.writelines(index_atomtypes)


In [64]:
def uniq_nonbon():

    with open ('gaff_nonbon.itp') as fhandle:
        lines = fhandle.readlines()

    lines = np.unique(lines)

    with open ('gaff_nonbon.itp', 'w') as fhandle:
        fhandle.writelines(lines)

In [65]:
def packmol_inp_gen(ion_names):
    packmol_input_string = """tolerance 2.0

# The file type of input and output files is PDB

filetype pdb

# The name of the output file

output ini.pdb

# 1000 water molecules and 400 urea molecules will be put in a box
# defined by the minimum coordinates x, y and z = 0. 0. 0. and maximum
# coordinates 40. 40. 40. That is, they will be put in a cube of side
# 40. (the keyword "inside cube 0. 0. 0. 40.") could be used as well.

structure {}.pdb
  number 800
  inside box 0. 0. 0. 80. 80. 80.
end structure

structure {}.pdb
  number 800
  inside box 0. 0. 0. 80. 80. 80.
end structure
""".format(ion_names[0], ion_names[1])
    with open ('ini.inp', 'w') as fhandle:
        fhandle.writelines(packmol_input_string)

In [66]:
if __name__ == "__main__":
    ion_names = []
    os.system('cp /scratch/xu.jiam/Gaussian/*py ./')
    os.system('cp /scratch/xu.jiam/Gaussian/*mdp ./')
    sims = ['CCN1C=C[N+](=C1)C', 'C(F)(F)(F)S(=NS(=O)(=O)C(F)(F)F)(=O)[O-]']
    for sim in sims:
        
        ion_name = smiles_to_iupac(sim)
        ion_name = re.sub('[ ~!@#$%^&*()_,=-]', '_', ion_name)
        ion_names.append(ion_name)
        smile_to_gauss(ion_name, sim)
        gen_submit(ion_name)
        os.system('sbatch handin_{}.slurm'.format(ion_name))
        
        #job_id = re.findall('[0-9]+', subprocess.check_output(["sbatch handin.slurm"], shell=True).decode())[0]  ## submit job and capture jobid.
        ### gaussian主程序、结束后
        #itp_unify(ion_name)
        #os.system("JOB_ID {} > gauss_job_id.log".format(job_id))
        
    top_file_gen(ion_names)
    uniq_nonbon()
    packmol_inp_gen(ion_names)
    
    #os.system('echo "{}" >> ini.inp'.format(packmol_input_string))
    os.system('cp /scratch/xu.jiam/packmol/packmol ./')
    os.system('./packmol < ini.inp')
    loadgro()
    os.system('mpirun -np 1 gmx_mpi editconf -f ini.pdb -o ini.gro -box 8 8 8')
    
    
    


    
    

cp: /scratch/xu.jiam/Gaussian/*py: No such file or directory
cp: /scratch/xu.jiam/Gaussian/*mdp: No such file or directory
1 molecule converted
1 molecule converted
sh: sbatch: command not found
1 molecule converted
1 molecule converted
sh: sbatch: command not found
cp: /scratch/xu.jiam/packmol/packmol: No such file or directory
sh: ./packmol: No such file or directory
sh: loadgro: command not found
sh: gmx: command not found


In [ ]:

#ion_name = "1_1_1_trifluoro_N__trifluoromethylsulfonyl_methanesulfonimidate"
ion_name = "1_hexyl_3_methylimidazol_3_ium"
itp_unify(ion_name)

In [43]:
np.unique(lines)

array(['[ atomtypes ]\n',
       'c2           6    12.010736    0.000000    A      3.399670E-01    3.598240E-01\n',
       'c3           6    12.010736    0.000000    A      3.399670E-01    4.577296E-01\n',
       'f            9    18.998403    0.000000    A      3.118146E-01    2.552240E-01\n',
       'h1           1     1.007941    0.000000    A      2.471353E-01    6.568880E-02\n',
       'h4           1     1.007941    0.000000    A      2.510553E-01    6.276000E-02\n',
       'h5           1     1.007941    0.000000    A      2.421463E-01    6.276000E-02\n',
       'hc           1     1.007941    0.000000    A      2.649533E-01    6.568880E-02\n',
       'na           7    14.006703    0.000000    A      3.249999E-01    7.112800E-01\n',
       'ne           7    14.006703    0.000000    A      3.249999E-01    7.112800E-01\n',
       'o            8    15.999405    0.000000    A      2.959922E-01    8.786400E-01\n',
       's6          16    32.064787    0.000000    A      3.5635

In [44]:
lines

['[ atomtypes ]\n',
 'c3           6    12.010736    0.000000    A      3.399670E-01    4.577296E-01\n',
 'f            9    18.998403    0.000000    A      3.118146E-01    2.552240E-01\n',
 's6          16    32.064787    0.000000    A      3.563595E-01    1.046000E+00\n',
 'ne           7    14.006703    0.000000    A      3.249999E-01    7.112800E-01\n',
 'sy          16    32.064787    0.000000    A      3.563595E-01    1.046000E+00\n',
 'o            8    15.999405    0.000000    A      2.959922E-01    8.786400E-01\n',
 'c3           6    12.010736    0.000000    A      3.399670E-01    4.577296E-01\n',
 'na           7    14.006703    0.000000    A      3.249999E-01    7.112800E-01\n',
 'c2           6    12.010736    0.000000    A      3.399670E-01    3.598240E-01\n',
 'hc           1     1.007941    0.000000    A      2.649533E-01    6.568880E-02\n',
 'h1           1     1.007941    0.000000    A      2.471353E-01    6.568880E-02\n',
 'h4           1     1.007941    0.000000    

In [ ]:
bondtypes

In [ ]:
for i in np.unique(np.array(atomtype_angles)[:,:4], axis=0):
    a = np.where(np.sum(np.array(atomtype_angles)[:,:4] == i, axis=1) == 4)
    print(np.hstack((i, np.average(np.array(np.array(atomtype_angles)[a[0]][:, -2:], dtype=float), axis=0))))

In [ ]:
=i

In [ ]:
b = np.average(np.array(np.array(atomtype_angles)[a[0]][:, -2:], dtype=float), axis=0)

In [ ]:
np.hstack((i,b))

In [ ]:
lines.pop(1)

In [ ]:
lines

In [ ]:
chg_1 = pd.read_csv('sol.chg', delimiter=' +', header=None, engine='python')
chg_2 = pd.read_csv('vac.chg', delimiter=' +', header=None, engine='python')
chg_avg = (chg_1.iloc[:,-1] + chg_2.iloc[:,-1])/2
chg_avg = chg_avg.round(decimals=4).astype(str)

In [ ]:
mol_output_lines = []
count = 0
for line in lines:
    if re.search(r'-?[0-9]+[.][0-9]+', line):
        ori_chg = re.findall(r'(-?[0-9]+[.][0-9]+)', line)[-1] #ori_chg, atom charge in original file.
        
        if len(ori_chg) == len(chg_avg[count]): ## same sign for new and old charge
            line = re.sub(ori_chg, str(chg_avg[count]), line)
        
        elif len(ori_chg) == len(chg_avg[count])+1: ## old negative, new positive charge
            line = re.sub(ori_chg, ' '+str(chg_avg[count]), line)
            
        elif len(ori_chg) == len(chg_avg[count])-1: ## old positive, new negative charge
            line = re.sub(' '+ori_chg, str(chg_avg[count]), line)
        
        count += 1
        mol_output_lines.append(line)
    else:
        mol_output_lines.append(line)
with open('{}_')

In [ ]:
chg_avg

In [ ]:
mol_output_lines

In [ ]:
import pandas as pd

In [ ]:
chg_1 = pd.read_csv('sol.chg', delimiter=' +', header=None, engine='python')
chg_2 = pd.read_csv('vac.chg', delimiter=' +', header=None, engine='python')
chg_avg = (chg_1.iloc[:,-1] + chg_2.iloc[:,-1])/2

In [ ]:
chg_avg

In [ ]:
chg_avg.iloc[0].astype(str)

In [193]:
def my_ceil(a, precision=0):
    return np.true_divide(np.ceil(a * 10**precision), 10**precision)

def my_floor(a, precision=0):
    return np.true_divide(np.floor(a * 10**precision), 10**precision)

In [223]:
import argparse
import pandas as pd
import regex as re
import numpy as np

ion_name = "1_1_1_trifluoro_N__trifluoromethylsulfonyl_methanesulfonimidate"
#ion_name = "1_hexyl_3_methylimidazol_3_ium"

chg_1 = pd.read_csv('sol_{}.chg'.format(ion_name), delimiter=' +', header=None, engine='python')
chg_2 = pd.read_csv('vac_{}.chg'.format(ion_name), delimiter=' +', header=None, engine='python')
chg_avg = 0.8*(chg_1.iloc[:,-1] + chg_2.iloc[:,-1])/2
chg_avg = chg_avg.round(decimals=6)

if sum(chg_avg) > 0:
    chg_avg.iloc[-1] = 0.8 - sum(chg_avg[:-1])
else:
    print(sum(chg_avg))
    chg_avg.iloc[-1] = - 0.8 - sum(chg_avg[:-1])

chg_avg.iloc[-1] = np.round(chg_avg.iloc[-1], decimals=6)
chg_avg = chg_avg.astype(str)
mol_output_lines = []
count = 0

with open('{}.mol2'.format(ion_name)) as ff:
    line = ff.readline()
    while line:
        if re.search(r'-?[0-9]+[.][0-9]+', line):
            ori_chg = re.findall(r'(-?[0-9]+[.][0-9]+)', line)[-1] #ori_chg, atom charge in original file.

            if len(ori_chg) == len(chg_avg[count])-2: ## same sign for new and old charge
                line = re.sub(ori_chg, str(chg_avg[count]), line)

                
            elif len(ori_chg) == len(chg_avg[count])-1: ## old negative, new positive charge
                line = re.sub(ori_chg, ' '+str(chg_avg[count]), line)

                
            elif len(ori_chg) == len(chg_avg[count])-3: ## old positive, new negative charge
                line = re.sub(' '+ori_chg, str(chg_avg[count]), line)


            count += 1
            mol_output_lines.append(line)
        elif "@<TRIPOS>UNITY_ATOM_ATTR" in line: 
            ## skip three lines for sobtop program.
            line = ff.readline()
            line = ff.readline()

        else:
            mol_output_lines.append(line)

        line = ff.readline()

with open('{}_resp2.mol2'.format(ion_name), 'w') as fhandle:
	fhandle.writelines(mol_output_lines)



-0.799999


In [222]:
chg_avg

0     -0.021285
1      0.094413
2     -0.075029
3     -0.014156
4     -0.015663
5     -0.191095
6      0.102109
7     -0.135537
8     -0.140903
9      0.176249
10    -0.083651
11    -0.262807
12     0.013392
13     0.013392
14     0.013392
15    -0.025913
16    -0.025913
17     0.032281
18     0.032281
19     0.014811
20     0.014811
21     0.051826
22     0.051826
23     0.116325
24     0.116325
25     0.189423
26     0.180245
27     0.180311
28     0.132847
29     0.132847
30     0.132846
Name: 4, dtype: object

In [71]:
np.hstack(([1,2], [3,4]))

array([1, 2, 3, 4])

In [204]:
sum(np.round(np.array(chg_avg),4))

0.7996

In [88]:
sum(chg_avg)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [76]:
a = np.array([[1,2,3],[3,4,5]])
a

array([[1, 2, 3],
       [3, 4, 5]])

In [90]:
np.column_stack([a,a])

array([[1, 2, 3, 1, 2, 3],
       [3, 4, 5, 3, 4, 5]])

In [93]:
a[:,[1,2]]

array([[2, 3],
       [4, 5]])